In [61]:
##Packages
import time
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import xarray as xr
import datetime as dt
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV,train_test_split 
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor,MLPClassifier
from sklearn.ensemble import RandomForestClassifier ,RandomForestRegressor
from sklearn.preprocessing import scale , StandardScaler
import tarfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import urllib.request
from tqdm import tqdm
import scipy
from scipy.spatial.distance import cdist

In [2]:
model = 'arome'  #weather model (arome or arpege)
param = "tp"   #parameter name in the file (cf cells below to know the parameter names -> exploration of metadata)
file_date = dt.datetime(2016, 2, 14) # Day example

In [44]:
fname = "C:/Users/moedj/DEFI-IA/forecast/train/2D_arome_2016/2D_%s_%s.nc" %(model,file_date.strftime('%Y%m%d'))
data = xr.open_dataset(fname)

**Alexia**

In [7]:
#coords_path = 'C:/Users/Alexia/Desktop/Data/Other/stations_coordinates.csv' #Perso

#xtrain_obs_path = 'C:/Users/Alexia/Desktop/Data/Train/X_station_train.csv' #Perso
#xtest_obs_path = 'C:/Users/Alexia/Desktop/Data/Test/X_station_test.csv' #Perso

#bltest_obs_path = 'C:/Users/Alexia/Desktop/Data/Test/Baselines/Baseline_observation_test.csv' #Perso
#bltrain_for_path = 'C:/Users/Alexia/Desktop/Data/Train/Baselines/Baseline_forecast_train.csv' #Perso
#bltest_for_path = 'C:/Users/Alexia/Desktop/Data/Test/Baselines/Baseline_forecast_test.csv' #Perso

#ytrain_path = 'C:/Users/Alexia/Desktop/Data/Train/Y_train.csv' #Perso

In [5]:
# ##Données
# coords_path = 'C:/Users/Alexia/Desktop/Data/Other/stations_coordinates.csv' #Perso
# xtrainS_path = 'C:/Users/Alexia/Desktop/Data/Train/X_station_train.csv' #Perso
# ytrain_path = 'C:/Users/Alexia/Desktop/Data/Train/Y_train.csv' #Perso
# xtestS_path = 'C:/Users/Alexia/Desktop/Data/Test/X_station_test.csv' #Perso
# bl_obsT_path = 'C:/Users/Alexia/Desktop/Data/Test/Baselines/Baseline_observation_test.csv' #Perso

**Dario**

In [18]:
#Données
coords_path = 'C:/Users/moedj/DEFI-IA/Data/Other/stations_coordinates.csv' #Perso

xtrain_obs_path = 'C:/Users/moedj/DEFI-IA/Data/Train/X_station_train.csv' #Perso
xtest_obs_path = 'C:/Users/moedj/DEFI-IA/Data/Test/X_station_test.csv' #Perso

bltest_obs_path = 'C:/Users/moedj/DEFI-IA/Data/Test/Baselines/Baseline_observation_test.csv' #Perso
bltrain_for_path = 'C:/Users/moedj/DEFI-IA/Data/Train/Baselines/Baseline_forecast_train.csv' #Perso
bltest_for_path = 'C:/Users/moedj/DEFI-IA/Data/Test/Baselines/Baseline_forecast_test.csv' #Perso

ytrain_path = 'C:/Users/moedj/DEFI-IA/Data/Train/Y_train.csv' #Perso

In [19]:
##Coords
coords = pd.read_csv(coords_path)
display(coords)
print('Missing Data :', coords.isna().sum().sum(), '\n', 'Shape is :', coords.shape)

,number_sta,lat,lon,height_sta
0,86118001,46.477,0.985,120.0
1,86149001,46.917,0.025,60.0
2,56081003,48.050,-3.660,165.0
3,53215001,47.790,-0.710,63.0
4,22135001,48.550,-3.380,148.0
...,...,...,...,...
320,86137003,47.035,0.098,96.0
321,86165005,46.412,0.841,153.0
322,86273001,46.464,1.042,121.0
323,91200002,48.526,1.993,116.0


Missing Data : 0 
 Shape is : (325, 4)


In [20]:
xtrain_obs = pd.read_csv(xtrain_obs_path,parse_dates=['date'],infer_datetime_format=True)
xtrain_obs['number_sta'] = xtrain_obs['number_sta'].astype('category')  
display(xtrain_obs)
print('Missing Data :', xtrain_obs.isna().sum().sum(), '\n', 'Shape is :', xtrain_obs.shape)

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


Missing Data : 6899007 
 Shape is : (4409474, 9)


In [21]:
xtrain_obs_ll = xtrain_obs.merge(coords, how='left', on='number_sta')
display(xtrain_obs_ll)

,number_sta,date,ff,t,td,hu,dd,precip,Id,lat,lon,height_sta
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,49.334,-0.431,2.0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,49.334,-0.431,2.0
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,49.334,-0.431,2.0
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,49.334,-0.431,2.0
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,49.334,-0.431,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19,49.108,1.831,126.0
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20,49.108,1.831,126.0
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21,49.108,1.831,126.0
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22,49.108,1.831,126.0


In [22]:
ytrain = pd.read_csv(ytrain_path, parse_dates=['date'], infer_datetime_format=True)
ytrain['number_sta'] = ytrain['number_sta'].astype('category')
display(ytrain)
print('Missing Data :', ytrain.isna().sum().sum(), '\n', 'Shape is :', ytrain.shape)

,date,number_sta,Ground_truth,Id
0,2016-01-02,14066001,3.4,14066001_0
1,2016-01-02,14126001,0.5,14126001_0
2,2016-01-02,14137001,3.4,14137001_0
3,2016-01-02,14216001,4.0,14216001_0
4,2016-01-02,14296001,13.3,14296001_0
...,...,...,...,...
183742,2017-12-31,86137003,5.0,86137003_729
183743,2017-12-31,86165005,3.2,86165005_729
183744,2017-12-31,86272002,1.8,86272002_729
183745,2017-12-31,91200002,1.6,91200002_729


Missing Data : 21640 
 Shape is : (183747, 4)


In [23]:
xtest_obs = pd.read_csv(xtest_obs_path,infer_datetime_format=True)
display(xtest_obs)
print('Missing Data :', xtest_obs.isna().sum().sum(), '\n', 'Shape is :', xtest_obs.shape)

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8
...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22


Missing Data : 3914287 
 Shape is : (2304802, 8)


In [24]:
bltest_obs = pd.read_csv(bltest_obs_path,infer_datetime_format=True)

In [25]:
bltrain_for = pd.read_csv(bltrain_for_path, infer_datetime_format=True)
bltest_for = pd.read_csv(bltest_for_path, infer_datetime_format=True)

## Données forecast

In [31]:
model = 'arome'
param = "tp"
file_date = dt.datetime(2016, 2, 14) 
X_fore_2D = "C:/Users/moedj/DEFI-IA/forecast/train/2D_arome_2016/2D_%s_%s.nc" %(model,file_date.strftime('%Y%m%d'))
data = xr.open_dataset(X_fore_2D)
data.to_netcdf("saved_on_disk.nc")

In [32]:
X_forecast_2D = xr.open_dataset("saved_on_disk.nc")

In [33]:
X_forecast_2D

<xarray.Dataset>
Dimensions:     (latitude: 227, longitude: 315, valid_time: 25, Id: 25)
Coordinates:
    time        datetime64[ns] ...
  * latitude    (latitude) float32 51.9 51.87 51.85 51.82 ... 46.3 46.27 46.25
  * longitude   (longitude) float32 -5.842 -5.817 -5.792 ... 1.958 1.983 2.008
  * valid_time  (valid_time) datetime64[ns] 2016-02-14 ... 2016-02-15
  * Id          (Id) object '2_43_0' '2_43_1' '2_43_2' ... '2_43_23' '2_43_24'
Data variables:
    ws          (valid_time, latitude, longitude) float32 ...
    p3031       (valid_time, latitude, longitude) float32 ...
    u10         (valid_time, latitude, longitude) float32 ...
    v10         (valid_time, latitude, longitude) float32 ...
    t2m         (valid_time, latitude, longitude) float32 ...
    d2m         (valid_time, latitude, longitude) float32 ...
    r           (valid_time, latitude, longitude) float32 ...
    tp          (valid_time, latitude, longitude) float32 ...
    msl         (valid_time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             lfpw
    GRIB_centreDescription:  French Weather Service - Toulouse
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             French Weather Service - Toulouse
    history:                 2021-09-29T14:50:11 GRIB to CDM+CF via cfgrib-0....

In [34]:
# ytrain_date = ytrain[(ytrain['date'] =='2016-02-14')]
# ytrain_date = ytrain_date.merge(coords, how='left', on='number_sta')
# display(ytrain_date)

In [35]:
#display(X_forecast_2D[param])

In [70]:
Obs = pd.DataFrame({'lon': xtrain_obs_ll.loc[:,"lon"], 'lat': xtrain_obs_ll.loc[:,"lat"]})
Obs

,lon,lat
0,-0.431,49.334
1,-0.431,49.334
2,-0.431,49.334
3,-0.431,49.334
4,-0.431,49.334
...,...,...
4409469,1.831,49.108
4409470,1.831,49.108
4409471,1.831,49.108
4409472,1.831,49.108


In [71]:
Forecast = pd.DataFrame({'lon': X_forecast_2D.longitude ,'lat': X_forecast_2D.latitude})
Forecast

ValueError: All arrays must be of the same length

In [74]:
cdist(Obs, Forecast)

NameError: name 'Forecast' is not defined